In [ ]:
from geopy.geocoders import Nominatim
import csv
import pandas as pd

In [ ]:
geolocator = Nominatim(user_agent="cholera_lab")

In [ ]:
geocode = geolocator.geocode("MORA+cameroon", country_codes="cmr")
print(geocode)

In [ ]:
country_dict = {
    "CMR": "Cameroon",
    "COD": "Congo",
    "COG": "Congo-Brazzaville",
    "ETH": "Ethiopia",
    "KEN": "Kenya",
    "MWI": "Malawi",
    "NER": "Niger",
    "SSD": "South Sudan",
    "TCD": "TChad",
}

In [ ]:
geocode_list = []
with open("manually_updated_admin_names.csv", "r") as file:
    reader = csv.reader(file)

    for row in reader:
        country = row[0]
        admin2 = row[1]
        str_to_query = admin2 + "+" + country_dict.get(country, "")
        geocode = geolocator.geocode(str_to_query, country_codes=country)
        address = None
        if geocode:
            address = geocode.address

        geocode_list.append(
            {"admin0": country, "admin2": admin2, "geocode match": address}
        )

In [ ]:
geocode_df = pd.DataFrame(geocode_list)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
for index, location in geocode_df["geocode match"].items():
    if location:
        address_parts = location.split(",")
        if len(address_parts) > 3:
            geocode_df.loc[index, "parsed admin2"] = address_parts[-3]
        elif len(address_parts) > 2:
            geocode_df.loc[index, "parsed admin2"] = address_parts[-2]
        if len(address_parts) == 1:
            geocode_df.loc[index, "parsed admin2"] = location
    else:
        geocode_df.loc[index, "parsed admin2"] = None

geocode_df.to_csv("matched_admin2_names.csv", index=False)
geocode_df